In [1]:
pip install langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python langchain-text-splitters gpt4all arxiv "gpt4all[cuda]"

  Using cached nvidia-cuda-runtime-cu12-0.0.1.dev5.tar.gz (4.3 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [17 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/private/var/folders/5j/s1w23gm133g69g8vtwz3hg480000gp/T/pip-install-9z_d2lj4/nvidia-cuda-runtime-cu12_d3f62f4e43f245bb815619619ab84145/setup.py", line 152, in <module>
          raise RuntimeError(open("ERROR.txt", "r").read())
      RuntimeError:
      ###########################################################################################
      The package you are trying to install is only a placeholder project on PyPI.org repository.
      This package is hosted on NVIDIA Python Package Index.
      
      This package can be installed as:
      ```
      $ pip install nvidia-

In [2]:
# will use llama3 model with Ollama
local_llm = 'llama3'

In [5]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ['OPENAI_API_KEY'] = ''

In [6]:
### Index

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

In [61]:
question = "llm agent memory"

In [62]:
### Router

from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# LLM
llm = ChatOllama(model=local_llm, format="json", temperature=0)

prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an expert at routing a
    user question to a vectorstore or web search. Use the vectorstore for questions on LLM  agents,
    prompt engineering, and adversarial attacks. You do not need to be stringent with the keywords
    in the question related to these topics. Otherwise, use web-search. Give a binary choice 'web_search'
    or 'vectorstore' based on the question. Return the a JSON with a single key 'datasource' and
    no premable or explanation. Question to route: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question"],
)

question_router = prompt | llm | JsonOutputParser()
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
answer = question_router.invoke({"question": question})

## router!
datasource = answer["datasource"]

In [50]:
def get_grader_prompt():
    grader_prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing relevance
        of a retrieved document to a user question. If the document contains keywords related to the user question,
        grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
        Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
        Provide the binary score as a JSON with a single key 'score' and no premable or explanation.
        <|eot_id|><|start_header_id|>user<|end_header_id|>
        Here is the retrieved document: \n\n {document} \n\n
        Here is the user question: {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
        """,
        input_variables=["question", "document"],
    )

    retrieval_grader = grader_prompt | llm | JsonOutputParser()
    docs = retriever.invoke(question)
    doc_txt = docs[1].page_content
    result_grader = retrieval_grader.invoke({"question": question, "document": doc_txt})
    if (result_grader["score"] == "yes"):
        return True
    
    return False

    # 1개 컨텐츠가 아니라 전체 컨텐츠를 보고싶을 때
    # result_docs = []
    # for doc in docs:
    #     result = retrieval_grader.invoke({"question": question, "document": doc.page_content})
    #     if (result['score'] == 'yes'):
    #         result_docs.__add__(doc)
        
    # return result_docs

In [51]:
print(get_grader_prompt())

False


In [52]:
### Generate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

def get_gernerated_prompt():
    
    # Prompt
    generated_prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks.
        Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
        Use three sentences maximum and keep the answer concise <|eot_id|><|start_header_id|>user<|end_header_id|>
        Question: {question}
        Context: {context}
        Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["question", "document"],
    )

    llm = ChatOllama(model=local_llm, temperature=0)

    # Chain
    rag_chain = generated_prompt | llm | StrOutputParser()

    # Run
    docs = retriever.invoke(question)
    generation = rag_chain.invoke({"context": docs, "question": question})
    return generation

In [53]:
generation_answer = get_gernerated_prompt()
print(generation_answer)

I don't know. The provided context does not mention Messi or any information about his current team or playing status. It appears to be a collection of documents discussing large language models, adversarial attacks, and their applications in various fields.


In [54]:
### Hallucination Grader

def get_hallucination_garder(generation):

    # LLM
    llm = ChatOllama(model=local_llm, format="json", temperature=0)

    # Prompt
    prompt = PromptTemplate(
        template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether
        an answer is grounded in / supported by a set of facts. Give a binary 'yes' or 'no' score to indicate
        whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a
        single key 'score' and no preamble or explanation. <|eot_id|><|start_header_id|>user<|end_header_id|>
        Here are the facts:
        \n ------- \n
        {documents}
        \n ------- \n
        Here is the answer: {generation}  <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["generation", "documents"],
    )

    hallucination_grader = prompt | llm | JsonOutputParser()
    return hallucination_grader.invoke({"documents": docs, "generation": generation})

In [33]:
result = get_hallucination_garder(generation_answer)
print(result)

{'score': 'no'}


In [55]:
### Answer Grader

def get_answer_grader(generation):
    # LLM
    llm = ChatOllama(model=local_llm, format="json", temperature=0)

    # Prompt
    prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether an
        answer is useful to resolve a question. Give a binary score 'yes' or 'no' to indicate whether the answer is
        useful to resolve a question. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
        <|eot_id|><|start_header_id|>user<|end_header_id|> Here is the answer:
        \n ------- \n
        {generation}
        \n ------- \n
        Here is the question: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["generation", "question"],
    )

    answer_grader = prompt | llm | JsonOutputParser()
    return answer_grader.invoke({"question": question, "generation": generation})

In [38]:
answer_garder = get_answer_grader(generation_answer)
print(answer_garder)

{'score': 'yes'}


In [56]:
from tavily import TavilyClient
tavily = TavilyClient(api_key='')

def search_query():
    
    # response = tavily.search(query=question, max_results=3)
    # context = [{"url": obj["url"], "content": obj["content"]} for obj in response['results']]

    # You can easily get search result context based on any max tokens straight into your RAG.
    # The response is a string of the context within the max_token limit.

    # response_context = tavily.get_search_context(query="Where does Messi play right now?", search_depth="advanced", max_tokens=500)

    # You can also get a simple answer to a question including relevant sources all with a simple function call:
    # You can use it for baseline
    return tavily.qna_search(query=question)


In [57]:
search_answer = search_query()
print(search_answer)

Lionel Messi currently plays for Inter Miami CF in Major League Soccer (MLS) in the United States.


In [63]:
print("datasource: ", datasource)

# vectorstore에 있는 내용과 유사하다면 document에서 검색 (related to index)
if (datasource == "vectorstore") :
    # retreve documents with grade
    

    # relavant condition
    is_relevant = get_grader_prompt()
    if (is_relevant):
        # generate answer
        generated_answer = get_gernerated_prompt()

        # hallucinations condition
        is_hallucination_answer = get_hallucination_garder(generated_answer)
        if (is_hallucination_answer["score"] == "no"):
            #answer question
            answer = get_answer_grader(generated_answer)
            if (answer["score"] == "yes"):
                # print
                print(generated_answer)

            else:
                print(search_query())
        else:
            print(search_query())
    else:
        print(search_query())
else:
    print(search_query())


datasource:  vectorstore
According to the provided context, Component Two of an LLM-powered autonomous agent system is Memory. This component includes Types of Memory and Maximum Inner Product Search (MIPS).
